In [165]:
import requests
import json
import pandas as pd

# map a list of Uniprot accessions to PDB ids
mapping_url = 'https://www.uniprot.org/uploadlists/'
params = {
'from':'ACC',
'to':'PDB_ID',
'format':'tab',
'query':'P13368 P20806 Q9UM73 P97793 Q17192'
}
header = {'User-Agent': 'Python'}
r = requests.get(mapping_url, params=params, headers=header)
ID_df = pd.read_csv(r.url, sep="\t", skiprows=1, header=None, names=["ACC", "PDB_ID"])
ID_df.dropna(inplace=True)

# get the corresponding Uniprot canonical AA sequences as a .fasta file
uniprot_url = "https://www.uniprot.org/uniprot/"
with open("canonical_input.fasta", "w") as f:
    for acc in ID_df["ACC"].values:
        query_url = uniprot_url + acc + ".fasta"
        r = requests.get(query_url, params=params, headers=header)
        f.write(r.text)

# align them using mafft
!rm -f aligned_canonical_input.fasta
!mafft --quiet --reorder --thread -1 canonical_input.fasta >aligned_canonical_input.fasta

In [166]:
RES_SHORT = ['arg', 'asn', 'asp', 'cys', 'gln', 'glu', 'his', 'ile', 'leu', 'lys', 'met', 'phe', 'pro', 'ser', 'thr',
             'trp', 'tyr', 'val', 'ala', 'gly']
RES_FULL = ['Arginine', 'Asparagine', 'Aspartate', 'Cysteine', 'Glutamate', 'Glutamine', 'Histidine', 'Isoleucine', 
            'Leucine', 'Lysine', 
            'Methionine', 'Phenylalanine', 'Proline', 'Serine', 'Threonine', 'Tryptophan', 'Tyrosine', 'Valine',
            'Alanine', 'Glycine']
res_dict = dict(zip([x.upper() for x in RES_SHORT], RES_FULL))

three_to_one = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K', 'TRP': 'W', 'PRO': 'P', 'THR': 'T',
                'ILE': 'I', 'ALA': 'A', 'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'MET': 'M',
                'GLU': 'E', 'ASN': 'N', 'TYR': 'Y', 'VAL': 'V'}
one_to_three_dict = dict([[v,k] for k,v in three_to_one.items()])

rota_seq_dict = {}
aa_seq_dict = {}
!rm -f polypeptide_input.fasta

df = pd.read_csv("Penultimate_2_Dunbrak.table", sep="\t", header=0, index_col=["Restype", "State"])

def url_response(url):
    r = requests.get(url=url)
    if r.status_code == 200:
        json_result = r.json()
        return json_result
    else:
        return None


def store_sequences(seq_id, rota_seq, aa_seq):
    rota_seq_dict[seq_id] = rota_seq
    aa_seq_dict[seq_id] = aa_seq
    # write 1-letter AA sequence as fasta
    with open("polypeptide_input.fasta", "a") as f:
        f.write(">" + seq_id + "\n" + "".join(map(str, [three_to_one[a] for a in aa_seq])) + "\n")
        
def get_sequences(pdb_id, entity_id, molecule):
    rama_search_url="https://www.ebi.ac.uk/pdbe/api/validation/rama_sidechain_listing/entry/"  + pdb_id
    results = url_response(rama_search_url)
    for entity in results[pdb_id.lower()]["molecules"]:
        if entity.get('entity_id') == entity_id:
            for chain in entity.get("chains"):
                chain_id = chain.get("chain_id")
                for model in chain.get("models"):
                    model_id = model.get("model_id")
                    seq_id = "_".join(map(str, [pdb_id, entity_id, chain_id, model_id])) 
                    rota_seq = []
                    aa_seq = []
                    for residue in model["residues"]:
                        aa_state = residue.get("residue_name")
                        if residue.get("residue_name") in ['ALA', 'GLY']:
                            rota_seq.append(aa_state)
                            aa_seq.append(aa_state)
                        else:
                            if residue.get("rama") == "Favored" or residue.get("rama") == "Allowed":
                                try:
                                    rota_seq.append(df.loc[res_dict[residue.get("residue_name")]].loc[residue.get("rota"), 
                                                                                       "Dunbrack_typical_state"])
                                    aa_seq.append(aa_state)
                                except:
                                    continue
                                    #aa_seq.append("-")
                                    #rota_seq.append("-")

                    store_sequences(seq_id, rota_seq, aa_seq)



# get the corresponding PDB entities 
# note that this results in one-to-many mapping of Uniprot ACC
entry_search_url = 'http://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/'
for pdb_id in ID_df["PDB_ID"].values:
    search_url = entry_search_url + pdb_id
    r = url_response(search_url)
    if r != None:
        for molecule in r[pdb_id.lower()]:
            # filter for polypeptides
            entity_id = molecule.get("entity_id")
            if "polypeptide" in molecule.get('molecule_type'):
                get_sequences(pdb_id, entity_id, molecule)


# align polypeptide sequences to the canonical sequence alignment
!rm -f aligned_polypeptide_input.fasta
!mafft --addfragments polypeptide_input.fasta --reorder --thread -1 aligned_canonical_input.fasta >aligned_polypeptide_input.fasta

OS = darwin
The number of physical cores =  4
nadd = 178
dndpre (aa) Version 7.407
alg=X, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

All-to-all alignment.
   56 / 57

##### writing hat3
pairlocalalign (aa) Version 7.407
alg=Y, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

nadd = 178
ppenalty_ex = -10
nthread = 4
blosum 62 / kimura 200
sueff_global = 0.100000
norg = 57
njobc = 58
Loading 'hat3' ... 
done.
Loading 'hat2n' (aligned sequences - new sequences) ... done.
Loading 'hat2i' (aligned sequences) ... done.
c77 / 178                    

Combining ..
   done.                      

   done.                       

addsingle (aa) Version 7.407
alg=A, model=BLOSUM62, 1.53, -0.00, +0.01, noshift, amax=0.0
0 thread(s)


Strategy:
 Multi-INS-fragment (Not tested.)
 ?

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has 

In [169]:
from Bio.SeqIO.FastaIO import SimpleFastaParser

one_rota_to_four_dict = {'1': 'VAL2', '0': 'VAL1', '2': 'VAL3', 'A': 'ALA', 'C': 'ARG2', 'B': 'ARG1', 'E': 'ASN1',
                         'D': 'ARG3', 'G': 'ASN3', 'F': 'ASN2', 'I': 'ASP2', 'H': 'ASP1', 'K': 'CYS1', 'J': 'ASP3',
                         'M': 'CYS3', 'L': 'CYS2', 'O': 'GLN2', 'N': 'GLN1', 'Q': 'GLU1', 'P': 'GLN3', 'S': 'GLU3',
                         'R': 'GLU2', 'U': 'HIS1', 'T': 'GLY', 'W': 'HIS3', 'V': 'HIS2', 'Y': 'ILE2', 'X': 'ILE1',
                         'Z': 'ILE3', 'a': 'LEU1', 'c': 'LEU3', 'b': 'LEU2', 'e': 'LYS2', 'd': 'LYS1', 'g': 'MET1',
                         'f': 'LYS3', 'i': 'MET3', 'h': 'MET2', 'k': 'PHE2', 'j': 'PHE1', 'm': 'PRO1', 'l': 'PHE3',
                         'o': 'SER1', 'n': 'PRO2', 'q': 'SER3', 'p': 'SER2', 's': 'THR2', 'r': 'THR1', 'u': 'TRP1',
                         't': 'THR3', 'w': 'TRP3', 'v': 'TRP2', 'y': 'TYR2', 'x': 'TYR1', 'z': 'TYR3'}

four_rota_to_one_dict = dict([[v,k] for k,v in one_rota_to_four_dict.items()])

!rm -f aligned_polypeptide_1-char.rotasequences.fasta

# parse alignment
with open('aligned_polypeptide_input.fasta') as fasta_file:
    seq_ids = []
    seqs = []
    for title, sequence in SimpleFastaParser(fasta_file):
        #get the sequence id and aligned sequence
        seq_ids.append(title.split(None, 1)[0])
        seqs.append(sequence)    
        s1 = pd.Series(seq_ids, name='seq_id')
        s2 = pd.Series(seqs, name='seq')
        fasta_df = pd.DataFrame(dict(seq_id=s1, seq=s2)).set_index(['seq_id'])

# map polypeptide rotasequences to aligned AA polypeptide sequences
for seq_id in rota_seq_dict.keys():
    rota_seq = rota_seq_dict[seq_id]
    aa_seq = aa_seq_dict[seq_id]
    aligned_aa_seq = list(fasta_df.loc[seq_id, 'seq'])
    aligned_rota_seq = ["-"]*len(aligned_aa_seq)
    
    j = 0
    for i,aligned_aa_state in enumerate(aligned_aa_seq):
        if aligned_aa_state != "-":  
            aligned_rota_seq[i] = rota_seq[j]
            j += 1
    with open("aligned_polypeptide_1-char.rotasequences.fasta", "a") as f:
        f.write(">" + seq_id + "\n" + \
                "".join(map(str, [four_rota_to_one_dict.get(r, "-") for r in aligned_rota_seq])) + "\n")